In [1]:

#Q1
from queue import PriorityQueue
from itertools import permutations

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f

def heuristic(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def best_first_search(maze, start, end):
    rows, cols = len(maze), len(maze[0])
    start_node = Node(start)
    frontier = PriorityQueue()
    frontier.put(start_node)
    visited = set()

    while not frontier.empty():
        current_node = frontier.get()
        current_pos = current_node.position

        if current_pos == end:
            path = []
            cost = 0
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
                cost += 1
            return path[::-1], cost - 1

        visited.add(current_pos)
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0 and new_pos not in visited:
                new_node = Node(new_pos, current_node)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, end)
                new_node.f = new_node.h
                frontier.put(new_node)
                visited.add(new_pos)

    return None, float('inf')

def find_optimal_path(maze, start, goals):

    shortest_path = None
    min_distance = float('inf')

    for perm in permutations(goals):
        total_path = []
        current_start = start
        total_distance = 0

        for goal in perm:
            path_segment, distance = best_first_search(maze, current_start, goal)
            if path_segment is None:
                break
            total_distance += distance
            total_path.extend(path_segment if not total_path else path_segment[1:])  # Avoid duplicate positions
            current_start = goal

        if path_segment is not None and total_distance < min_distance:
            min_distance = total_distance
            shortest_path = total_path

    return shortest_path, min_distance


maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goals = [(2, 3), (4, 4), (1, 4)]


path, cost = find_optimal_path(maze, start, goals)

if path:
    print("Shortest path covering all goals:", path)
    print("Total path cost (steps taken):", cost)
else:
    print("No path found covering all goals.")


Shortest path covering all goals: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]
Total path cost (steps taken): 10


In [4]:
#Q2

import heapq
import random
import time

class AStar:
    def __init__(self, graph, heuristic):
        self.graph = graph
        self.heuristic = heuristic
        self.g_costs = {}
        self.came_from = {}
        self.frontier = []
        self.visited = set()

    def a_star(self, start, goal):
        self.g_costs = {start: 0}
        self.came_from = {start: None}
        heapq.heappush(self.frontier, (self.heuristic[start], start))

        while self.frontier:
            _, current_node = heapq.heappop(self.frontier)

            if current_node in self.visited:
                continue

            self.visited.add(current_node)


            if current_node == goal:
                path = self.reconstruct_path(goal)
                print(f"\nGoal reached! Optimal Path: {path}")
                print(f"Total Cost: {self.g_costs[goal]}")
                return path


            for neighbor, cost in self.graph[current_node].items():
                new_g_cost = self.g_costs[current_node] + cost
                f_cost = new_g_cost + self.heuristic[neighbor]

                if neighbor not in self.g_costs or new_g_cost < self.g_costs[neighbor]:
                    self.g_costs[neighbor] = new_g_cost
                    self.came_from[neighbor] = current_node
                    heapq.heappush(self.frontier, (f_cost, neighbor))

            self.update_edge_costs()

        print("\nNo path found")
        return None

    def reconstruct_path(self, goal):
        path = []
        current_node = goal
        while current_node is not None:
            path.append(current_node)
            current_node = self.came_from.get(current_node)
        return path[::-1]

    def update_edge_costs(self):
        node = random.choice(list(self.graph.keys()))
        if self.graph[node]:
            neighbor = random.choice(list(self.graph[node].keys()))
            change = random.choice([-2, -1, 1, 2])
            self.graph[node][neighbor] = max(1, self.graph[node][neighbor] + change)
            print(f"\nDynamic Cost Update: Edge ({node} → {neighbor}) changed to {self.graph[node][neighbor]}")


graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 3},
    'D': {'B': 2, 'G': 1},
    'E': {'B': 5, 'G': 2, 'H': 3},
    'F': {'C': 3, 'H': 6},
    'G': {'D': 1, 'E': 2, 'I': 3},
    'H': {'E': 3, 'F': 6, 'I': 1},
    'I': {'G': 3, 'H': 1}
}


heuristic = {
    'A': 7, 'B': 6, 'C': 6,
    'D': 5, 'E': 5, 'F': 5,
    'G': 3, 'H': 2, 'I': 0
}


astar_solver = AStar(graph, heuristic)


print("\nRunning Adaptive A* Search...\n")
optimal_path = astar_solver.a_star('A', 'I')



Running Adaptive A* Search...


Dynamic Cost Update: Edge (G → D) changed to 1

Dynamic Cost Update: Edge (A → B) changed to 2

Dynamic Cost Update: Edge (D → B) changed to 1

Dynamic Cost Update: Edge (B → E) changed to 7

Goal reached! Optimal Path: ['A', 'B', 'D', 'G', 'I']
Total Cost: 7


In [8]:
#Q3
import heapq
import math

class DeliveryScheduler:
    def __init__(self, delivery_points, time_windows, start_point):
        self.delivery_points = delivery_points
        self.time_windows = time_windows
        self.start_point = start_point

    def calculate_distance(self, point1, point2):
        x1, y1 = self.delivery_points[point1]
        x2, y2 = self.delivery_points[point2]
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

    def prioritize_delivery(self, location, current_time):
        window_start, window_end = self.time_windows[location]
        urgency_factor = max(0, window_end - current_time)
        return self.calculate_distance(self.start_point, location) + urgency_factor * 2

    def generate_optimized_route(self):
        priority_queue = []
        visited = set()
        delivery_route = []
        current_location = self.start_point
        current_time = 0

        heapq.heappush(priority_queue, (0, current_location, current_time))  # Push starting point

        while priority_queue:
            _, current_location, current_time = heapq.heappop(priority_queue)

            if current_location in visited:
                continue

            visited.add(current_location)
            delivery_route.append((current_location, current_time))

            remaining_points = [loc for loc in self.delivery_points if loc not in visited]
            for next_point in remaining_points:
                travel_time = self.calculate_distance(current_location, next_point)
                estimated_arrival = current_time + travel_time

                if self.time_windows[next_point][0] <= estimated_arrival <= self.time_windows[next_point][1]:
                    priority_score = self.prioritize_delivery(next_point, estimated_arrival)
                    heapq.heappush(priority_queue, (priority_score, next_point, estimated_arrival))

        return delivery_route

delivery_locations = {
    'A': (0, 0),
    'B': (2, 3),
    'C': (5, 4),
    'D': (6, 1)
}
time_windows = {
    'A': (0, 10),
    'B': (3, 8),
    'C': (5, 12),
    'D': (7, 15)
}

scheduler = DeliveryScheduler(delivery_locations, time_windows, 'A')
optimized_route = scheduler.generate_optimized_route()

print("\nOptimized Delivery Route:")
for location, time in optimized_route:
    print(f"{location}  Time: {round(time, 2)}")



Optimized Delivery Route:
A  Time: 0
B  Time: 3.61
C  Time: 6.77
D  Time: 9.93


In [11]:
#Q4

import heapq
import random
import time

class TrafficOptimizedRoute:
    def __init__(self, road_network, live_traffic):
        self.road_network = road_network
        self.live_traffic = live_traffic

    def calculate_heuristic(self, current, destination):
        x1, y1 = location_coordinates[current]
        x2, y2 = location_coordinates[destination]
        return ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5

    def get_current_travel_time(self, from_node, to_node):
        return self.live_traffic.get((from_node, to_node), self.live_traffic.get((to_node, from_node), None))

    def find_fastest_route(self, start, destination):
        print(f"\ Searching for the **fastest route** from {start} to {destination}...\n")
        priority_queue = []
        heapq.heappush(priority_queue, (0, start))

        came_from = {start: None}
        travel_time_cost = {start: 0}

        while priority_queue:
            _, current_location = heapq.heappop(priority_queue)

            print(f"At {current_location} (Elapsed time: {travel_time_cost[current_location]} mins)")

            if current_location == destination:
                break

            for neighbor, base_travel_time in self.road_network.get(current_location, []):
                updated_travel_time = self.get_current_travel_time(current_location, neighbor)
                if not updated_travel_time:
                    updated_travel_time = base_travel_time

                new_cost = travel_time_cost[current_location] + updated_travel_time

                if neighbor not in travel_time_cost or new_cost < travel_time_cost[neighbor]:
                    travel_time_cost[neighbor] = new_cost
                    priority = new_cost + self.calculate_heuristic(neighbor, destination)
                    heapq.heappush(priority_queue, (priority, neighbor))
                    came_from[neighbor] = current_location
                    print(f"  {current_location} → {neighbor} | Cost: {updated_travel_time} mins")

            time.sleep(1)

        path = self.build_route(came_from, start, destination)

        if path:
            print(f"\nFastest Route Found: {' → '.join(path)}")
            print(f"Total Travel Time: {travel_time_cost[destination]} mins\n")
        else:
            print("\nNo route available.\n")

        return path

    def build_route(self, came_from, start, destination):
        path = []
        current = destination
        while current != start:
            path.append(current)
            current = came_from.get(current)
            if current is None:
                return []
        path.append(start)
        path.reverse()
        return path

    def update_live_traffic(self):
        print("\nReal-Time Traffic Update Incoming\n")
        for road in self.live_traffic:
            self.live_traffic[road] = random.randint(1, 10)
            print(f"  Traffic Change: {road[0]} → {road[1]} now takes {self.live_traffic[road]} mins")
        time.sleep(2)

location_coordinates = {
    'A': (0, 0), 'B': (2, 4), 'C': (3, 1), 'D': (5, 5), 'E': (7, 3)
}

road_network = {
    'A': [('B', 4), ('C', 2)],
    'B': [('A', 4), ('D', 5), ('E', 10)],
    'C': [('A', 2), ('D', 8)],
    'D': [('B', 5), ('C', 8), ('E', 2)],
    'E': [('B', 10), ('D', 2)]
}

traffic_conditions = {
    ('A', 'B'): 4, ('A', 'C'): 2, ('B', 'D'): 5, ('B', 'E'): 10,
    ('C', 'D'): 8, ('D', 'E'): 2
}

city_navigator = TrafficOptimizedRoute(road_network, traffic_conditions)

start_point, destination = 'A', 'E'
print("\nStarting Initial Route Search...")
initial_route = city_navigator.find_fastest_route(start_point, destination)

city_navigator.update_live_traffic()

print("\nRecalculating Fastest Route After Traffic Update...\n")
updated_route = city_navigator.find_fastest_route(start_point, destination)



Starting Initial Route Search...
\ Searching for the **fastest route** from A to E...

At A (Elapsed time: 0 mins)
  A → B | Cost: 4 mins
  A → C | Cost: 2 mins
At C (Elapsed time: 2 mins)
  C → D | Cost: 8 mins
At B (Elapsed time: 4 mins)
  B → D | Cost: 5 mins
  B → E | Cost: 10 mins
At D (Elapsed time: 9 mins)
  D → E | Cost: 2 mins
At E (Elapsed time: 11 mins)

Fastest Route Found: A → B → D → E
Total Travel Time: 11 mins


Real-Time Traffic Update Incoming

  Traffic Change: A → B now takes 2 mins
  Traffic Change: A → C now takes 8 mins
  Traffic Change: B → D now takes 4 mins
  Traffic Change: B → E now takes 5 mins
  Traffic Change: C → D now takes 1 mins
  Traffic Change: D → E now takes 9 mins

Recalculating Fastest Route After Traffic Update...

\ Searching for the **fastest route** from A to E...

At A (Elapsed time: 0 mins)
  A → B | Cost: 2 mins
  A → C | Cost: 8 mins
At B (Elapsed time: 2 mins)
  B → D | Cost: 4 mins
  B → E | Cost: 5 mins
At E (Elapsed time: 7 mins)

F